## 使用GPU

### 方式1

将他们加载到GPU上，使用.cuda()
- 网络模型
- 数据
- 损失函数

具体见代码

In [10]:
import  torchvision
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import torch
import time

GPU训练

可以看到时间比cpu快：

cuda training
----epoch 0 starting----
time:1.1624798774719238

batch 100---loss:2.2960407733917236
time:2.374920606613159

batch 200---loss:2.2844889163970947
time:3.6774864196777344

batch 300---loss:2.2557263374328613
time:5.16971492767334



In [14]:

##准备数据集
train_data = torchvision.datasets.CIFAR10("../../dataset/",train=True,transform=torchvision.transforms.ToTensor(),download=True)
test_data = torchvision.datasets.CIFAR10("../../dataset/",train=False,transform=torchvision.transforms.ToTensor(),download=True)

#数据集长度
train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练数据集的长度为{},测试数据集的长度为{}".format(train_data_size,test_data_size))

##加载数据集
train_dataloader = DataLoader(train_data,batch_size=64)
test_dataloader = DataLoader(test_data,batch_size=64)

##搭建神经网络
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,5,1,2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10)
#             nn.Linear(64,10),
#             nn.Softmax(dim=1)     ###其实softmax加上去结果很不好，反而原来的好
        )
    def forward(self,x):
        x = self.model(x)
        return x

model  = Model()
###############################################
if torch.cuda.is_available():
    print("cuda training")
    model = model.cuda()
###############################################
##损失函数
loss_fn = nn.CrossEntropyLoss()
###############################################
if torch.cuda.is_available():
        loss_fn = loss_fn.cuda()
###############################################
##优化器
learning_rate=1e-2
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

##设置训练网络的一些参数
##记录训练的次数
total_train_step= 0
##记录测试的次数
total_test_step =0
##训练的轮数
epoch=10
##添加tensorboard
start_time = time.time()
for i in range(epoch):
    print("----epoch {} starting----".format(i))
    model.train()
    ##训练步骤开始
    for data in train_dataloader:
        imgs,targets = data
        ###############################################
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            targets = targets.cuda()
        ###############################################
        outputs = model(imgs)
        loss = loss_fn(outputs,targets)

        ##优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step+=1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print("time:{}".format(end_time-start_time))
            print("batch {}---loss:{}".format(total_train_step,loss.item()))
    
    ##测试
    model.eval()
    total_test_loss = 0
    total_accuarcy = 0
    
    with torch.no_grad():##让网络模型无梯度
        for data in test_dataloader:
            imgs,targets = data
            ###############################################
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                targets = targets.cuda()
             ################################################   
            outputs = model(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss+=loss
            accuarcy = (outputs.argmax(1)==targets).sum()
            total_accuarcy+=accuarcy
    print("avarage testing loss:{}".format(total_test_loss/test_data_size))
    acc = total_accuarcy / test_data_size
    print("acc:{}".format(acc))
    total_test_step+=1
    

Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为50000,测试数据集的长度为10000
cuda training
----epoch 0 starting----
time:1.1624798774719238
batch 100---loss:2.2960407733917236
time:2.374920606613159
batch 200---loss:2.2844889163970947
time:3.6774864196777344
batch 300---loss:2.2557263374328613
time:5.16971492767334
batch 400---loss:2.1544976234436035


KeyboardInterrupt: 

CPU训练

time:5.044811487197876

batch 100---loss:2.2861838340759277
time:11.217432022094727

batch 200---loss:2.282048463821411
time:17.956006288528442

batch 300---loss:2.2508111000061035
time:24.85852336883545

batch 400---loss:2.1681175231933594
time:31.897176265716553

In [15]:

##准备数据集
train_data = torchvision.datasets.CIFAR10("../../dataset/",train=True,transform=torchvision.transforms.ToTensor(),download=True)
test_data = torchvision.datasets.CIFAR10("../../dataset/",train=False,transform=torchvision.transforms.ToTensor(),download=True)

#数据集长度
train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练数据集的长度为{},测试数据集的长度为{}".format(train_data_size,test_data_size))

##加载数据集
train_dataloader = DataLoader(train_data,batch_size=64)
test_dataloader = DataLoader(test_data,batch_size=64)

##搭建神经网络
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,5,1,2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10)
#             nn.Linear(64,10),
#             nn.Softmax(dim=1)     ###其实softmax加上去结果很不好，反而原来的好
        )
    def forward(self,x):
        x = self.model(x)
        return x

model  = Model()
# if torch.cuda.is_available():
#     print("cuda training")
#     model = model.cuda()
##损失函数
loss_fn = nn.CrossEntropyLoss()

# if torch.cuda.is_available():
#         loss_fn = loss_fn.cuda()

##优化器
learning_rate=1e-2
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

##设置训练网络的一些参数
##记录训练的次数
total_train_step= 0
##记录测试的次数
total_test_step =0
##训练的轮数
epoch=10
##添加tensorboard
start_time = time.time()
for i in range(epoch):
    print("----epoch {} starting----".format(i))
    model.train()
    ##训练步骤开始
    for data in train_dataloader:
        imgs,targets = data
#         if torch.cuda.is_available():
#             imgs = imgs.cuda()
#             targets = targets.cuda()
        outputs = model(imgs)
        loss = loss_fn(outputs,targets)

        ##优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step+=1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print("time:{}".format(end_time-start_time))
            print("batch {}---loss:{}".format(total_train_step,loss.item()))
    
    ##测试
    model.eval()
    total_test_loss = 0
    total_accuarcy = 0
    
    with torch.no_grad():##让网络模型无梯度
        for data in test_dataloader:
            imgs,targets = data
#             if torch.cuda.is_available():
#                 imgs = imgs.cuda()
#                 targets = targets.cuda()
            outputs = model(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss+=loss
            accuarcy = (outputs.argmax(1)==targets).sum()
            total_accuarcy+=accuarcy
    print("avarage testing loss:{}".format(total_test_loss/test_data_size))
    acc = total_accuarcy / test_data_size
    print("acc:{}".format(acc))
    total_test_step+=1
    

Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为50000,测试数据集的长度为10000
----epoch 0 starting----
time:5.044811487197876
batch 100---loss:2.2861838340759277
time:11.217432022094727
batch 200---loss:2.282048463821411
time:17.956006288528442
batch 300---loss:2.2508111000061035
time:24.85852336883545
batch 400---loss:2.1681175231933594
time:31.897176265716553
batch 500---loss:2.035565137863159
time:38.80791449546814
batch 600---loss:2.0351269245147705
time:45.61942219734192
batch 700---loss:2.014039993286133


KeyboardInterrupt: 

### 通用方式2

首先定义device

然后使用.to(device)在：

    - 模型
    - 数据，标注
    - 损失函数
    
注意数据和标注需要赋值，而模型和损失函数并不是必须：

    如 model.to(device)即可，不用model = model.to(device)；但是数据和label记得要

####  device定义方式
1. cpu
2. GPU:cuda  / 多张GPU
3. 通用定义

In [ ]:
device = torch.device("cpu")

In [ ]:
device = torch.device("cuda")

device = torch.device("cuda:0")##有多张显卡时对应特定显卡序号

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### 具体代码测试

In [23]:
device = torch.device("cuda:0")


##准备数据集
train_data = torchvision.datasets.CIFAR10("../../dataset/",train=True,transform=torchvision.transforms.ToTensor(),download=True)
test_data = torchvision.datasets.CIFAR10("../../dataset/",train=False,transform=torchvision.transforms.ToTensor(),download=True)

#数据集长度
train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练数据集的长度为{},测试数据集的长度为{}".format(train_data_size,test_data_size))

##加载数据集
train_dataloader = DataLoader(train_data,batch_size=128)
test_dataloader = DataLoader(test_data,batch_size=128)

##搭建神经网络
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,1,2),
            nn.MaxPool2d(2),
           
            nn.Conv2d(32,64,5,1,2),
            nn.MaxPool2d(2),
#             nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10)
#             nn.Linear(64,10),
#             nn.Softmax(dim=1)     ###其实softmax加上去结果很不好，反而原来的好
        )
    def forward(self,x):
        x = self.model(x)
        return x

model  = Model()
model  = model.to(device)
##损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)
##优化器
learning_rate=1e-2
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

##设置训练网络的一些参数
##记录训练的次数
total_train_step= 0
##记录测试的次数
total_test_step =0
##训练的轮数
epoch=10
##添加tensorboard
start_time = time.time()
for i in range(epoch):
    print("----epoch {} starting----".format(i))
    model.train()
    ##训练步骤开始
    for data in train_dataloader:
        imgs,targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = model(imgs)
        loss = loss_fn(outputs,targets)

        ##优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step+=1
        if total_train_step % 200 == 0:
            end_time = time.time()
            print("time:{}".format(end_time-start_time))
            print("batch {}---loss:{}".format(total_train_step,loss.item()))
    
    ##测试
    model.eval()
    total_test_loss = 0
    total_accuarcy = 0
    
    with torch.no_grad():##让网络模型无梯度
        for data in test_dataloader:
            imgs,targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss+=loss
            accuarcy = (outputs.argmax(1)==targets).sum()
            total_accuarcy+=accuarcy
    print("avarage testing loss:{}".format(total_test_loss/test_data_size))
    acc = total_accuarcy / test_data_size
    print("acc:{}".format(acc))
    total_test_step+=1
    
torch.save(model,"../../model_10.pth")
    

Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为50000,测试数据集的长度为10000
----epoch 0 starting----
time:25.88600754737854
batch 200---loss:2.289736270904541
avarage testing loss:0.017700864002108574
acc:0.15429998934268951
----epoch 1 starting----
time:45.108107566833496
batch 400---loss:2.2158942222595215
time:58.37624406814575
batch 600---loss:2.0101733207702637
avarage testing loss:0.015590372495353222
acc:0.28839999437332153
----epoch 2 starting----
time:73.59836173057556
batch 800---loss:2.040374994277954
time:86.32196187973022
batch 1000---loss:1.9103964567184448
avarage testing loss:0.01465497724711895
acc:0.35109999775886536
----epoch 3 starting----
time:99.13157439231873
batch 1200---loss:1.853210210800171
time:108.50026369094849
batch 1400---loss:1.736275553703308
avarage testing loss:0.01404686737805605
acc:0.37389999628067017
----epoch 4 starting----
time:121.19837546348572
batch 1600---loss:1.7123537063598633
time:130.442889213562
batch 1800

### 查看机器上显卡

In [20]:
!nvidia-smi

Thu Oct 13 11:28:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.41       Driver Version: 471.41       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   67C    P0    N/A /  N/A |    619MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------